In [19]:
import control as ct
import numpy as np
from scipy.linalg import solve_lyapunov, solve_continuous_are

np.set_printoptions(3, linewidth=500)

In [ ]:
g = 9.81
m = 1.0
Ixx = 0.015
Iyy = 0.015
Izz = 0.03
l = 0.225
Ka = 2e-7
Km = 2e-9
 
psi = 0.0
 
a_sub = np.array([[0, 0, 0, g*np.sin(psi), g*np.cos(psi), 0],
                          [0, 0, 0, -g*np.cos(psi), g*np.sin(psi), 0]])
a_sub = np.vstack((a_sub, np.zeros((4, 6))))


A = np.block([[np.zeros((6,6)), np.eye(6)],
                [a_sub, np.zeros((6,6))]])


b_sub = np.array([[1/m, 0, 0, 0],
                    [0, 1/Ixx, 0, 0],
                    [0, 0, 1/Iyy, 0],
                    [0, 0, 0, 1/Izz]]) 

# b_sub = np.array([[Ka/m, Ka/m, Ka/m, Ka/m],
#             [0, -Ka*l/Ixx, 0, Ka*l/Ixx],
#             [Ka*l/Iyy, 0, -Ka*l/Iyy, 0],
#             [Km/Izz, -Km/Izz, Km/Izz, -Km/Izz]]) # For direct rpm
        
B = np.vstack((np.zeros((8, 4)), b_sub))

Q = np.eye(12)*600
desired_poles = -np.linspace(1, 12, 12)
K = ct.place(A, B, desired_poles)

Kr_ref_gain = np.linalg.pinv(B) @ (A - B @ K)
Am = A - B@K
Bm = np.copy(B)

P = solve_lyapunov(Am.T, -Q)

Kx = -K.T
Kr = np.eye(4) 


print("\nKx:")
print(Kx)

print("\nKr_ref_gain:")
print(Kr_ref_gain)

print("\nP:")
print(P)



Kx:
[[ 2.277e-02  1.762e-01 -3.929e-01 -1.213e-03]
 [ 1.451e-02  1.135e+00 -1.761e-01  8.347e-04]
 [-5.237e+01 -7.864e-06  8.577e-05  2.250e-01]
 [-1.675e-02 -3.308e+00  2.317e-01 -1.640e-03]
 [ 5.665e-02  2.317e-01 -2.332e+00 -3.531e-03]
 [ 7.499e+00 -6.975e-05 -1.942e-04 -1.579e+00]
 [ 2.351e-02  1.227e-01 -5.812e-01 -1.384e-03]
 [ 9.395e-03  1.098e+00 -1.227e-01  7.320e-04]
 [-1.498e+01 -9.897e-07  1.051e-05  3.000e-02]
 [-9.136e-04 -3.871e-01  1.287e-02 -1.055e-04]
 [ 3.754e-03  1.287e-02 -3.329e-01 -2.448e-04]
 [ 9.999e-01 -8.849e-06 -2.501e-05 -4.505e-01]]

Kr_ref_gain:
[[ 2.277e-02  1.451e-02 -5.237e+01 -1.675e-02  5.665e-02  7.499e+00  2.351e-02  9.395e-03 -1.498e+01 -9.136e-04  3.754e-03  9.999e-01]
 [ 1.762e-01  1.135e+00 -7.864e-06 -3.308e+00  2.317e-01 -6.975e-05  1.227e-01  1.098e+00 -9.897e-07 -3.871e-01  1.287e-02 -8.849e-06]
 [-3.929e-01 -1.761e-01  8.577e-05  2.317e-01 -2.332e+00 -1.942e-04 -5.812e-01 -1.227e-01  1.051e-05  1.287e-02 -3.329e-01 -2.501e-05]
 [-1.213e-0